# Long Short-Term Memory

In [537]:
#Import des Apple Datensatzes
%store -r df 

In [538]:
# Import der benötigten Bibliotheken für Neuronale Netze
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

from scikeras.wrappers import KerasRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [539]:
# Aufteilung in Trainings- und Testdaten
X = df.drop(columns = ["close_next"])
y = df["close_next"]

# Trainings- und Testdaten
X_train_test, X_test, y_train_test, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Trainings- und Validierungsdaten
X_train, X_val, y_train, y_val = train_test_split(X_train_test, y_train_test, test_size = 0.25, random_state = 42)

# Daten Normalisieren mithilfe des MinMaxScaler
sc = MinMaxScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)
X_val_scaled = sc.transform(X_val)

In [540]:
# Reshape der Eingabedaten, damit das LSTM-Modell trainiert werden kann
X_train_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], 1, X_train_scaled.shape[1])
X_test_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], 1, X_test_scaled.shape[1])
X_val_reshaped = X_val_scaled.reshape(X_val_scaled.shape[0], 1, X_val_scaled.shape[1])

### LSTM-Modell:

In [592]:
# Funktion zur Erstellung des Keras Modells
def create_model():
    model = Sequential()
    
    #Erste LSTM-Schicht
    model.add(LSTM(units = 256, return_sequences = True, input_shape = (1, 7)))
    model.add(Dropout(0.2))
    
    #Zweite LSTM-Schicht
    model.add(LSTM(units = 512, return_sequences = True))
    model.add(Dropout(0.2))
    
    #dritte LSTM-Schicht
    model.add(LSTM(units = 512, return_sequences = True))
    model.add(Dropout(0.2))
    
    #vierte LSTM-Schicht
    model.add(LSTM(units = 512, return_sequences = True))
    model.add(Dropout(0.2))
    
    #fünfte LSTM-Schicht
    model.add(LSTM(units = 256))
    model.add(Dropout(0.2))
    
    #Dense-Schichten
    model.add(Dense(units = 64, activation = "relu", kernel_regularizer = regularizers.l2(0.05)))
    model.add(Dense(units = 32, activation = "relu", kernel_regularizer = regularizers.l2(0.05)))
    model.add(Dense(units = 16, activation = "relu", kernel_regularizer = regularizers.l2(0.05)))
    model.add(Dense(units = 8, activation = "relu", kernel_regularizer = regularizers.l2(0.05)))
    model.add(Dense(units = 1))
    
    model.compile(loss = "mean_squared_error", optimizer = "adam")
    return model

# Erstellen eines KerasRegressor
lstm = KerasRegressor(model = create_model, verbose = 0)

# Fit
lstm.fit(X_train_reshaped, y_train, epochs = 200, batch_size = 64)

# Bewertung auf Basis der Testdaten
lstm_test_score = lstm.score(X_test_reshaped, y_test)
print("test R^2 score LSTM-Modell: {:.2f}%".format(lstm_test_score * 100))

# Vorhersage auf Basis der Testdaten
y_pred = lstm.predict(X_test_reshaped)

# mean_absolute_error
mae = mean_absolute_error(y_test, y_pred)

# mean_squared_error
mse = mean_squared_error(y_test, y_pred)

# Root mean squared error
rmse = np.sqrt(mse)

# mean absolute percentage error
mape = mean_absolute_percentage_error(y_test, y_pred)

print("Mean Absolute Error (MAE) LSTM-Modell: {:.2f}".format(mae))
print("Mean Squared Error (MSE) LSTM-Modell: {:.2f}".format(mse))
print("Root Mean Squared Error (RMSE) LSTM-Modell: {:.2f}".format(rmse))
print("Mean Absolute Percentage Error (MAPE) LSTM-Modell: {:.2f}".format(mape))

test R^2 score LSTM-Modell: 99.93%
Mean Absolute Error (MAE) LSTM-Modell: 0.45
Mean Squared Error (MSE) LSTM-Modell: 0.93
Root Mean Squared Error (RMSE) LSTM-Modell: 0.97
Mean Absolute Percentage Error (MAPE) LSTM-Modell: 0.60
